## 特徵工程（Feature Engineering）

本章節將根據前一階段 EDA 的觀察，執行所有特徵的處理，包括：

- 欄位清理與刪除（如 id）
- 數值衍生特徵（如 BMI、Duration²）
- 分箱變數建立（AgeGroup、Temp_Binary）
- 類別交叉特徵（Sex_AgeGroup、HR_TempRisk）
- 偏態轉換（log1p / Yeo-Johnson）
- ~~離群值處理（IQR）~~  
> 離群值處理（如 IQR）雖曾嘗試納入，但刪除後導致模型表現下降。  
> 考量目前使用的 XGBoost 屬於樹模型，對離群值較不敏感，故最終**保留所有樣本資料**不做移除。

**套件匯入與用途說明**

本章將使用以下模組完成特徵工程處理流程：

- **NumPy / Pandas**：基礎數據處理與資料結構操作
- **StandardScaler**：將數值特徵標準化（均值 = 0，標準差 = 1），使模型學習更穩定
- **KMeans**：將樣本依據相似性進行分群，產生新的群集特徵（如 `KMeans_cluster`）
- **skew（偏態）**：量化欄位的對稱程度，用於判斷是否需要進行轉換處理
- **PowerTransformer**：對偏態特徵執行 **Yeo-Johnson 轉換**，提升模型穩定性與泛化能力
- **warnings**：忽略不必要的 sklearn 警告訊息，使 Notebook 輸出更乾淨

In [29]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from scipy.stats import skew
from sklearn.preprocessing import PowerTransformer

In [31]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

## 特徵擴充與分群處理（Feature Expansion & Clustering）

本章節針對訓練與測試資料進行完整的特徵工程操作，主要目的為：

- 擴充出具生理意義的數值與分類欄位
- 透過群組分箱、交叉特徵加強資訊密度
- 建立 KMeans 分群特徵補充隱含樣本分佈

---

### 步驟 1：定義生理推估與分箱函式

| 函式名稱          | 功能說明                                   |
|-------------------|--------------------------------------------|
| `duration_group`  | 將運動時長分為 short / medium / long       |
| `age_group`       | 將年齡分為 Child / Teen / YoungAdult 等群組 |
| `compute_hr_max`  | 使用 Gellish 公式計算最大心率 HR_max       |
| `estimate_met`    | 根據 HR 百分比估算 MET 活動強度            |
| `compute_bmr`     | 使用 Mifflin-St Jeor 公式推估基礎代謝率     |

---

### 步驟 2：主特徵工程函式 `apply_feature_engineering(df)`

此函式將對每筆樣本套用以下處理：

#### 數值衍生特徵
| 欄位名稱           | 說明                             |
|--------------------|----------------------------------|
| `Duration_squared` | 運動時長平方，建模非線性關係    |
| `log1p_Duration`   | 對時長做 log1p 緩解偏態          |
| `BMI`              | 體重 / 身高²，健康風險衡量        |

#### 風險/群組分箱（分類特徵）
| 欄位名稱           | 說明                              |
|--------------------|-----------------------------------|
| `Temp_Binary`      | 體溫二元風險分箱（是否高於 39.5）   |
| `HeartRate_Binary` | 心率二元風險分箱（是否高於 99.5）   |
| `Duration_group`   | 運動時長分箱：short / medium / long |
| `AgeGroup`         | 年齡分群（Child ~ Senior）         |

#### 生理推估特徵
| 欄位名稱         | 說明                                               |
|------------------|----------------------------------------------------|
| `HR_max`         | 推估最大心率                                       |
| `HR_percent`     | 實際心率佔最大心率比例                             |
| `MET`            | 根據 HR 百分比推估的活動強度指數                  |
| `Calories_Est`   | 預估消耗卡路里（MET × 體重 × 時長）                 |
| `BMR`            | 推估基礎代謝率（性別 + 體重 + 年齡 + 身高）         |
| `TDEE_Est`       | 推估每日總消耗能量（BMR + 消耗熱量）                |

#### 交叉特徵（複合變數）

- **類別交叉特徵**（可用來進行 One-Hot Encoding）
  - `Sex_AgeGroup`
  - `Age_TempRisk`
  - `Age_MET`
  - `Sex_TempRisk`
  - `HR_DurationRisk`

- **數值交互特徵**
  - `HR_Temp`：心率 × 體溫
  - `Weight_Duration`：體重 × 運動時長
  - `HR_per_Age`：心率 / 年齡（避免除以 0）

---

### 步驟 3：KMeans 分群（選用）

若有提供 `StandardScaler` 與 `KMeans 模型`（僅在訓練資料擬合），則：

- 以 `['Duration', 'Heart_Rate', 'Body_Temp', 'BMI', 'HR_percent', 'MET']` 為分群依據
- 加入新欄位 `KMeans_cluster`，反映隱含樣本族群

---

### 步驟 4：刪除不再使用的原始欄位

```python
for col in ['Height', 'Weight']:
    if col in df.columns:
        df.drop(columns=col, inplace=True)

In [33]:
# =========================
# 分箱與生理推估函式定義
# =========================

def duration_group(duration):
    """將運動時長分為 short / medium / long"""
    if duration < 10:
        return "short"
    elif 10 <= duration <= 20:
        return "medium"
    else:
        return "long"

def age_group(age):
    """將年齡分群為 Child / Teen / YoungAdult / MiddleAge / Senior"""
    if age <= 12:
        return "Child"
    elif age <= 18:
        return "Teen"
    elif age <= 29:
        return "YoungAdult"
    elif age <= 59:
        return "MiddleAge"
    else:
        return "Senior"

def compute_hr_max(age):
    """使用 Gellish 公式計算最大心率 HRmax = 207 - 0.7 * 年齡"""
    return 207 - 0.7 * age

def estimate_met(hr_pct):
    """根據最大心率百分比估算 MET 活動強度指數"""
    if hr_pct < 0.5:
        return 2.0  # 輕度活動
    elif hr_pct < 0.7:
        return 4.5  # 中等活動
    elif hr_pct < 0.85:
        return 7.0  # 劇烈活動
    else:
        return 9.5  # 高強度運動

def compute_bmr(row):
    """
    使用 Mifflin-St Jeor 公式計算基礎代謝率（BMR）
    男性：10W + 6.25H − 5A + 5；女性：10W + 6.25H − 5A − 161
    """
    sex = str(row['Sex']).lower()
    if sex == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    else:
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161


# =========================
# 主特徵工程函式（KMeans模型為選填）
# =========================

def apply_feature_engineering(df, scaler=None, kmeans_model=None):
    """
    對資料集執行完整特徵工程：
    - 分箱處理
    - 數值與交叉特徵
    - 生理推估欄位（MET, HR_max）
    - KMeans 分群（若提供模型）
    """

    # 數值衍生欄位
    df['Duration_squared'] = df['Duration'] ** 2
    df['log1p_Duration'] = np.log1p(df['Duration'])
    df['BMI'] = df['Weight'] / ((df['Height'] / 100) ** 2)

    # 二元風險分箱
    df['Temp_Binary'] = np.where(df['Body_Temp'] <= 39.5, 0, 1)
    df['HeartRate_Binary'] = np.where(df['Heart_Rate'] <= 99.5, 0, 1)

    # 分箱欄位
    df['Duration_group'] = df['Duration'].apply(duration_group)
    df['AgeGroup'] = df['Age'].apply(age_group)

    # 生理推估特徵
    df['HR_max'] = df['Age'].apply(compute_hr_max)
    df['HR_percent'] = df['Heart_Rate'] / df['HR_max']
    df['MET'] = df['HR_percent'].apply(estimate_met)
    df['Calories_Est'] = df['MET'] * df['Weight'] * (df['Duration'] / 60)

    # 類別型交叉特徵
    df['Sex_AgeGroup'] = df['Sex'].astype(str) + "_" + df['AgeGroup']
    df['Age_TempRisk'] = df['AgeGroup'].astype(str) + "_" + df['Temp_Binary'].astype(str)
    df['Age_MET'] = df['AgeGroup'].astype(str) + "_" + df['MET'].astype(str)
    df['Sex_TempRisk'] = df['Sex'].astype(str) + "_" + df['Temp_Binary'].astype(str)
    df['HR_DurationRisk'] = df['HeartRate_Binary'].astype(str) + "_" + df['Duration_group']

    # 數值交叉特徵
    df['HR_Temp'] = df['Heart_Rate'] * df['Body_Temp']
    df['Weight_Duration'] = df['Weight'] * df['Duration']
    df['HR_per_Age'] = df['Heart_Rate'] / (df['Age'] + 1)

    # 推估能量消耗
    df['BMR'] = df.apply(compute_bmr, axis=1)
    df['TDEE_Est'] = df['BMR'] + df['Calories_Est']

    # 若有提供 KMeans 模型與標準化器，就新增分群特徵
    if (scaler is not None) and (kmeans_model is not None):
        cluster_cols = ['Duration', 'Heart_Rate', 'Body_Temp', 'BMI', 'HR_percent', 'MET']
        X_cluster = scaler.transform(df[cluster_cols])
        df['KMeans_cluster'] = kmeans_model.predict(X_cluster)

    return df


# =========================
# 建立 KMeans 模型（只能用於訓練資料）
# =========================

def fit_kmeans_model(df, n_clusters=4):
    """
    使用訓練資料建立 KMeans 分群模型與對應標準化器
    """
    cluster_cols = ['Duration', 'Heart_Rate', 'Body_Temp', 'BMI', 'HR_percent', 'MET']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[cluster_cols])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    kmeans.fit(X_scaled)
    return scaler, kmeans


# =========================
# 主流程：先做特徵 ➝ 建 KMeans ➝ 加入分群 ➝ 刪除欄位
# =========================

# 第一步：先進行特徵工程（此時尚未做分群）
train_df = apply_feature_engineering(train_df)
test_df = apply_feature_engineering(test_df)

# 第二步：建立 KMeans 模型（需要用上 BMI、MET 等前面步驟產生的欄位）
scaler, kmeans_model = fit_kmeans_model(train_df)

# 第三步：重新執行特徵工程，這次加入 KMeans_cluster
train_df = apply_feature_engineering(train_df, scaler, kmeans_model)
test_df = apply_feature_engineering(test_df, scaler, kmeans_model)

# 第四步：刪除不再需要的原始欄位（Height、Weight）
for col in ['Height', 'Weight']:
    if col in train_df.columns:
        train_df.drop(columns=col, inplace=True)
        test_df.drop(columns=col, inplace=True)

In [34]:
train_df.dtypes # 檢查現有特徵欄位及其對應型態

id                    int64
Sex                  object
Age                   int64
Duration            float64
Heart_Rate          float64
Body_Temp           float64
Calories            float64
Duration_squared    float64
log1p_Duration      float64
BMI                 float64
Temp_Binary           int64
HeartRate_Binary      int64
Duration_group       object
AgeGroup             object
HR_max              float64
HR_percent          float64
MET                 float64
Calories_Est        float64
Sex_AgeGroup         object
Age_TempRisk         object
Age_MET              object
Sex_TempRisk         object
HR_DurationRisk      object
HR_Temp             float64
Weight_Duration     float64
HR_per_Age          float64
BMR                 float64
TDEE_Est            float64
KMeans_cluster        int32
dtype: object

## 數值特徵偏態處理（Skewness Correction）

在模型訓練前，若數值特徵具有高度偏態（skewness），可能影響模型對極端值的學習效果。  
為此，我們對偏態進行修正，使特徵分佈更接近常態，提升模型穩定性。

---

### 處理目標

- 修正偏態值超出 [-0.5, 0.5] 的欄位
- 避免高偏態造成模型權重偏移或過度擬合
- 對數轉換與 Yeo-Johnson 轉換為主要修正方法

---

### 處理邏輯與方法

1. 篩選數值特徵欄位，排除下列欄位：
   - 目標變數 `Calories`
   - 已處理特徵 `log1p_Duration`、`Duration_squared`、`Calories_EST`
2. 計算各欄位偏態值（skewness）
3. 根據偏態數值分類處理：
   - 偏態 > 0.5 且皆為正數：使用 `np.log1p`
   - 偏態 > 0.5 含有零或負數：使用 `PowerTransformer(method='yeo-johnson')`
   - 偏態 < -0.5：使用 `PowerTransformer(method='yeo-johnson')`
4. 建立轉換前後的偏態比較表


In [46]:
# ==============================================
# 數值特徵偏態處理（Skewness Correction）
# ==============================================

# 所有數值特徵（排除 binary 與分類欄位）
numerical_features = [
    "Age", "Duration", "Heart_Rate", "Body_Temp",
    "Duration_squared", "log1p_Duration", "BMI",
    "HR_max", "HR_percent", "MET", "Calories_EST", "HR_Temp",
    "Weight_Duration", "HR_per_Age", "BMR", "TDEE_Est", "Calories"
]

# 排除不需轉換的欄位（目標值、已處理過欄位、人工高偏態特徵）
excluded_cols = ["Calories", "log1p_Duration", "Duration_squared", "Calories_EST"]
numeric_cols = [col for col in numerical_features if col not in excluded_cols]

# 計算原始偏態
original_skewness = train_df[numeric_cols].skew().sort_values(ascending=False)

# 複製資料（保留原始資料）
train_df_transformed = train_df.copy()
test_df_transformed = test_df.copy()

# 建立轉換器記錄字典
transformers = {}

# 執行偏態處理
for col in numeric_cols:
    if train_df[col].nunique() <= 1:
        continue  # 常數欄位略過

    # 偏態 > 0.5：右偏
    if original_skewness[col] > 0.5:
        if (train_df[col] > 0).all():
            # 對數轉換（只適用於正數）
            train_df_transformed[col] = np.log1p(train_df[col])
            test_df_transformed[col] = np.log1p(test_df[col])
        else:
            pt = PowerTransformer(method='yeo-johnson')
            train_df_transformed[col] = pt.fit_transform(train_df[[col]])
            test_df_transformed[col] = pt.transform(test_df[[col]])
            transformers[col] = pt

    # 偏態 < -0.5：左偏 ➝ Yeo-Johnson
    elif original_skewness[col] < -0.5:
        pt = PowerTransformer(method='yeo-johnson')
        train_df_transformed[col] = pt.fit_transform(train_df[[col]])
        test_df_transformed[col] = pt.transform(test_df[[col]])
        transformers[col] = pt

# 檢查轉換後的偏態
transformed_skewness = train_df_transformed[numeric_cols].skew().sort_values(ascending=False)

# 建立前後對照表
skew_df = pd.DataFrame({
    'Original Skew': original_skewness,
    'Transformed Skew': transformed_skewness
}).sort_values(by='Original Skew', ascending=False)

# 顯示偏態變化
print(skew_df)

                 Original Skew  Transformed Skew
HR_per_Age            0.670685          0.261040
Age                   0.436397          0.436397
Weight_Duration       0.372994          0.372994
HR_percent            0.311953          0.311953
TDEE_Est              0.107471          0.107471
BMR                   0.067535          0.067535
Duration              0.026259          0.026259
BMI                  -0.003291         -0.003291
Heart_Rate           -0.005668         -0.005668
HR_Temp              -0.076337         -0.076337
HR_max               -0.436397         -0.436397
MET                  -0.705669         -0.105841
Body_Temp            -1.022361         -0.165926


## 類別特徵編碼（One-Hot Encoding）

在完成特徵擴充與偏態處理後，需將類別型欄位轉為機器學習模型可處理的格式。這裡我們使用 One-Hot Encoding，將每個類別值轉為獨立的二元欄位。

---

### 編碼處理步驟

1. 複製偏態處理後的資料集（避免覆蓋原始）
2. 將 `KMeans_cluster`（原為 int）轉為類別型（字串）
3. 對 9 個類別欄位執行 One-Hot 編碼
4. 對齊訓練集與測試集欄位，補齊測試集中缺少的欄位（補 0）

---

### 類別特徵欄位清單

| 欄位名稱           | 說明                       |
|--------------------|----------------------------|
| `Sex`              | 性別                       |
| `Duration_group`   | 運動時長分箱               |
| `AgeGroup`         | 年齡群分箱                 |
| `Sex_AgeGroup`     | 性別 + 年齡群交叉特徵      |
| `Age_TempRisk`     | 年齡群 + 體溫風險交叉特徵  |
| `Age_MET`          | 年齡群 + MET 交叉特徵      |
| `Sex_TempRisk`     | 性別 + 體溫風險交叉特徵     |
| `HR_DurationRisk`  | 心率風險 + 時長交叉特徵     |
| `KMeans_cluster`   | KMeans 分群結果（數值轉字串） |


In [48]:
# 複製偏態處理後的資料（train_df_transformed 是你偏態處理 + 清理離群值的資料）
cleaned_train_df = train_df_transformed.copy()
cleaned_test_df = test_df_transformed.copy()

# 確保 KMeans_cluster 是類別型態（轉為字串）
cleaned_train_df["KMeans_cluster"] = cleaned_train_df["KMeans_cluster"].astype(str)
cleaned_test_df["KMeans_cluster"] = cleaned_test_df["KMeans_cluster"].astype(str)

# 原本的類別欄位，加上 KMeans_cluster 共 9 個
categorical_cols = [
    "Sex",                # 性別
    "Duration_group",     # 運動時長分箱
    "AgeGroup",           # 年齡群分箱
    "Sex_AgeGroup",       # 性別 + 年齡群交叉
    "Age_TempRisk",       # 年齡群 + 體溫風險交叉
    "Age_MET",            # 年齡群 + MET 等級交叉
    "Sex_TempRisk",       # 性別 + 體溫風險交叉
    "HR_DurationRisk",    # 心率風險 + 運動時長交叉
    "KMeans_cluster"      # 新增：分群特徵
]

# 對訓練與測試資料進行 One-Hot 編碼
cleaned_train_df = pd.get_dummies(cleaned_train_df, columns=categorical_cols)
cleaned_test_df = pd.get_dummies(cleaned_test_df, columns=categorical_cols)

# 對齊欄位：以 train 欄位為主，test 補缺失欄位（填 0）
cleaned_train_df, cleaned_test_df = cleaned_train_df.align(
    cleaned_test_df,
    join="left",   # 以 train 欄位為準
    axis=1,
    fill_value=0   # 測試集若缺欄位，用 0 補上
)

## XGBoost 模型訓練與 K-Fold 驗證（XGBoost Model Training and K-Fold Evaluation）

本區段將使用先前經由自動調參（Optuna）挑選出的最佳超參數組合進行 XGBoost 模型訓練。  
這組參數已在多次嘗試不同調參策略與集成模型（例如 Stacking, Bagging）後，依據 RMSLE 評分表現證實效果最穩定且優異，故後續皆統一使用此組參數進行建模與預測。  

流程包含以下步驟：
1. 建立 RMSLE 評估函式  
2. 載入最佳參數組  
3. 使用全部資料訓練最終模型  
4. 使用 5-Fold 交叉驗證評估模型穩定性  
5. 輸出 submission.csv 檔案

In [62]:
import numpy as np
import pandas as pd
import joblib
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from tqdm import tqdm

# =======================
# 1. 資料準備
# =======================
X = cleaned_train_df.drop(columns=['Calories', 'id'])
y = cleaned_train_df['Calories']
X_test = cleaned_test_df.drop(columns=['id', 'Calories'], errors='ignore')

# =======================
# 2. RMSLE 評估函式（可選）
# =======================
def rmsle_score(y_true, y_pred):
    y_true = np.clip(y_true, 0, None)
    y_pred = np.clip(y_pred, 0, None)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# =======================
# 3. 指定最佳參數（不調參）
# =======================
best_params = {
    'learning_rate': 0.020713786607324515,
    'max_depth': 10,
    'n_estimators': 2884,
    'colsample_bytree': 0.5193209214878505,
    'gamma': 6.806895577188799,
    'min_child_weight': 1,
    'reg_lambda': 8.432490165527739,
    'tree_method': 'hist',
    'device': 'cuda',
    'eval_metric': 'mae',
    'random_state': 42
}

# =======================
# 4. 訓練模型（使用全部資料）
# =======================
print("使用指定參數訓練 XGBoost 模型...")
model = XGBRegressor(**best_params)
for _ in tqdm(range(1), desc="XGBoost Training", ncols=100):
    model.fit(X, y)

from sklearn.model_selection import KFold

# =======================
# K-Fold RMSLE 評估
# =======================
print("\n🔍 使用 K-Fold 交叉驗證評估 RMSLE...")

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmsle_scores = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    model_fold = XGBRegressor(**best_params)
    model_fold.fit(X_train_fold, y_train_fold)

    y_pred_fold = model_fold.predict(X_valid_fold)
    fold_rmsle = rmsle_score(y_valid_fold, y_pred_fold)
    rmsle_scores.append(fold_rmsle)

    print(f"Fold {fold+1} RMSLE：{fold_rmsle:.5f}")

# 平均與標準差
mean_rmsle = np.mean(rmsle_scores)
std_rmsle = np.std(rmsle_scores)

print(f"\n平均 RMSLE（K-Fold）：{mean_rmsle:.5f} ± {std_rmsle:.5f}")

# 可選：儲存模型
joblib.dump(model, "final_xgboost_from_best_params.pkl")

# =======================
# 5. 預測並產出 submission.csv
# =======================
print("\n開始產出 submission.csv...")
preds = model.predict(X_test)
preds = np.clip(preds, 0, None)

submission = pd.DataFrame({
    'id': cleaned_test_df['id'],
    'Calories': preds
})
submission.to_csv("submission.csv", index=False)
print("submission.csv 已產出")

使用指定參數訓練 XGBoost 模型...


XGBoost Training: 100%|███████████████████████████████████████████████| 1/1 [00:57<00:00, 57.12s/it]



🔍 使用 K-Fold 交叉驗證評估 RMSLE...
Fold 1 RMSLE：0.06026
Fold 2 RMSLE：0.06071
Fold 3 RMSLE：0.06002
Fold 4 RMSLE：0.06113
Fold 5 RMSLE：0.06053

平均 RMSLE（K-Fold）：0.06053 ± 0.00038

開始產出 submission.csv...
submission.csv 已產出
